In [11]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load dos dados do ficheiro JSON e preparação dos dados para treino e teste
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    captured_data = data['capturedData']

    accelerometerXAxis = [entry['accelerometerXAxis'] for entry in captured_data]
    accelerometerYAxis = [entry['accelerometerYAxis'] for entry in captured_data]
    accelerometerZAxis = [entry['accelerometerZAxis'] for entry in captured_data]
    gyroscopeXAxis = [entry['gyroscopeXAxis'] for entry in captured_data]
    gyroscopeYAxis = [entry['gyroscopeYAxis'] for entry in captured_data]
    gyroscopeZAxis = [entry['gyroscopeZAxis'] for entry in captured_data]

    # criar dataFrame 
    df = pd.DataFrame({
        'accelerometerXAxis': accelerometerXAxis,
        'accelerometerYAxis': accelerometerYAxis,
        'accelerometerZAxis': accelerometerZAxis,
        'gyroscopeXAxis': gyroscopeXAxis,
        'gyroscopeYAxis': gyroscopeYAxis,
        'gyroscopeZAxis': gyroscopeZAxis
    })

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df.values)

    sequence_length = 11
    sequences = []
    for i in range(len(scaled_data) - sequence_length):
        sequences.append(scaled_data[i:i+sequence_length])

    sequences = np.array(sequences)

    # conjuntos de treino e teste
    X = sequences[:, :-1]
    y = sequences[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, scaler

#treinar o modelo LSTM
def build_and_train_model(X_train, X_test, y_train, y_test):
    # construção do LSTM
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(X_train.shape[2])) 
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

    return model

# prever os valores e mostrar a comparação
def predict_and_compare(model, X_test, y_test, scaler):
    y_pred = model.predict(X_test)

    y_pred_inverse = scaler.inverse_transform(y_pred)
    y_test_inverse = scaler.inverse_transform(y_test)

    comparison_df = pd.DataFrame({'Real': y_test_inverse.flatten(), 'Predicted': y_pred_inverse.flatten()})

    difference = np.abs(comparison_df['Real'] - comparison_df['Predicted'])
    comparison_df['Absolute Difference'] = difference

    return comparison_df
def main():
    datasets = [
        # BMW e36 datasets 10m
        r'..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\normal\BMW e36\acceleration 0m 10m normal-23-04-2023-19-30-17.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\slow\BMW e36\acceleration 0m 10m slow-23-04-2023-19-19-20.json',

    ]


    #comparação dos resultados
    models = []
    scalers = []
    comparison_dfs = []

    for dataset in datasets:
        print("Load e treino do modelo para:", dataset)
        X_train, X_test, y_train, y_test, scaler = load_dataset(dataset)
        model = build_and_train_model(X_train, X_test, y_train, y_test)
        models.append(model)
        scalers.append(scaler)

    for i, model in enumerate(models):
        print("\n\nResultados para o Dataset:", datasets[i])
        X_train, X_test, y_train, y_test, scaler = load_dataset(datasets[i])
        comparison_df = predict_and_compare(model, X_test, y_test, scalers[i])
        comparison_dfs.append(comparison_df)
        print(comparison_df)  
        print("\n-----------------------------\n")  



if __name__ == "__main__":
    main()


Load e treino do modelo para: ..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json
Epoch 1/100


C:\Users\root3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.2447 - val_loss: 0.3108
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.2175 - val_loss: 0.2886
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.1914 - val_loss: 0.2676
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.1664 - val_loss: 0.2479
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.1423 - val_loss: 0.2292
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.1192 - val_loss: 0.2116
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0972 - val_loss: 0.1953
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0766 - val_loss: 0.1802
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0575 - val_loss: 0.1669
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0403 - val_loss: 0.1555
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0255 - val_loss: 0.1465
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0138 - val_loss: 0.1406
Epo

C:\Users\root3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.1895 - val_loss: 0.1903
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.1667 - val_loss: 0.1805
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.1455 - val_loss: 0.1715
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.1258 - val_loss: 0.1631
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.1074 - val_loss: 0.1556
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0903 - val_loss: 0.1491
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0744 - val_loss: 0.1437
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0598 - val_loss: 0.1397
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0466 - val_loss: 0.1373
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0349 - val_loss: 0.1369
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0249 - val_loss: 0.1388
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0166 - val_loss: 0.1433
Epo

C:\Users\root3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.3808 - val_loss: 0.2384
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.3495 - val_loss: 0.2149
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.3203 - val_loss: 0.1930
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.2929 - val_loss: 0.1726
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.2670 - val_loss: 0.1535
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.2425 - val_loss: 0.1358
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.2193 - val_loss: 0.1195
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.1973 - val_loss: 0.1046
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.1765 - val_loss: 0.0912
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.1569 - val_loss: 0.0794
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.1388 - val_loss: 0.0695
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.1223 - val_loss: 0.0617
Epo